In [42]:
import numpy as np
import pandas as pd
import math
import re

In [43]:
FILENAME="test.txt"
FILENAME="input.txt"
DIAG=False

In [44]:
# OK
def parseFile(fileName):   
    with open(fileName) as file:
        lines =  file.readlines()
    
    counts = []
    for i, line in enumerate(lines):
        lines[i] = (line[:-1] if line[-1]=="\n" else line)
        counts.append([int(x) for x in lines[i].split()[1].split(",")])
        lines[i] = lines[i].split()[0]
    
    return (lines,counts)
    
    


In [45]:
# dirty trick to guarantee line starts with ".". However caused problems when moving to partB, as this 
# needs to be done AFTER multiplying the line!
def touchUpMap(lines):
    lines2 = []
    for line in lines:
        lines2.append("."+line)
    return lines2

In [46]:
#OK
STR_F = "[\?\#]"
STR_B = "[\.\?]"

In [47]:
#OK
def isStoppingCondition(rowLine,rowCount):
    bStop = False
    res = 0
    
    if len(rowCount) == 0:# must end with empty chars only
        if re.fullmatch(STR_B+"*",rowLine) is None: 
            bStop = True
            res = 0 
        else:
            bStop = True
            res = 1
    
    if len(rowLine) == 0 and len(rowCount) > 0:
        res =  0
        bStop = True
    if DIAG: print("STOPPING?",rowLine, rowCount,bStop,res)
    return (bStop,res)

In [48]:
#OK
def hasLiteralAfterBlanks(rowLine,litLen):
    bFoundCorrect = False
    lPosition = -1
      
    str_search = STR_F+"{"+str(litLen)+"}"            
    re_res = re.search(str_search,rowLine[1:])
    #print(rowLine,litLen,re_res)
    # have we found literal at all?
    if re_res is None: # CANNOT FIND THE GROUP
        bFoundCorrect = False
    else:
        lPosition = re_res.start()+1 #first match
        #is it preceeded by only "blanks?"
        if re.fullmatch(STR_B+"+",rowLine[:lPosition]) is None: 
            bFoundCorrect = False
        else:
            bFoundCorrect = True
        
        
    return (bFoundCorrect,lPosition)

In [49]:
DICT_CACHE = {}

In [50]:
#OK
def dealWithRow(rowLine,rowCount):
    key = (rowLine[:],str(rowCount[:]))
    if key in DICT_CACHE:         
        return DICT_CACHE[key]
    
    if DIAG: print("A>","-"+rowLine+"-",rowCount)
    # 1. Is this a stopping situation
    bStop, tmp_res = isStoppingCondition(rowLine,rowCount)
    if bStop: return tmp_res
    
    # 2. Check if can find the next group - and if it starts with blank space
    bFoundLiteral,lStartF = hasLiteralAfterBlanks(rowLine,rowCount[0])
    
    if not bFoundLiteral: 
        if DIAG: print("NotFound")
        return 0
    
    if DIAG:print("lStartF",lStartF,"rowCount[0]",rowCount[0])
    # 3. We found literal, let's a. recurse b. consider starting "?"    
    res = 0
    rowLine = rowLine[lStartF:]
    
    # 3a. Do recursive search
    res += dealWithRow(rowLine[rowCount[0]:],rowCount[1:])
    
    # 3b. If match starts with "?" then this could be treated as a blank
    if len(rowLine)>0 and rowLine[0] == "?":
        #print("Hop",rowLine)
        rowLine = "." + rowLine[1:]
        tempRes = dealWithRow(rowLine,rowCount)
        res += tempRes
        #print(rowLine,rowCount,tempRes)
        #rowLine=rowLine[1:]
    
    
    DICT_CACHE[key]=res
    
    return res

In [51]:
assert dealWithRow(".#",[1]) == 1
assert dealWithRow(".?..",[1]) == 1
assert dealWithRow(".??...?##.",[ 1,3]) == 2
assert dealWithRow("????",[1]) == 3
assert dealWithRow(".??..??...?##.",[ 1,1,3]) == 4
assert dealWithRow(".??...?##.",[ 1,3]) == 2
assert dealWithRow(".??..??..",[ 1,1]) == 4
assert dealWithRow(".#.#.?.",[1,1]) == 1
assert dealWithRow('.???.####..',[1,4]) == 3
assert dealWithRow( '.????.######..#####.',[1,6,5]) == 4

ss = '.????.#...#...?????.#...#...?????.#...#...?????.#...#...?????.#...#...'
cc = [4, 1, 1, 4, 1, 1, 4, 1, 1, 4, 1, 1, 4, 1, 1]
assert dealWithRow(ss,cc) == 16

ss = '.????.#...#...?.????.#...#...?.????.#...#...?.????.#...#...?.????.#...#...'
cc = [4, 1, 1, 4, 1, 1, 4, 1, 1, 4, 1, 1, 4, 1, 1]
assert dealWithRow(ss,cc) == 1

In [52]:
MULTIPL=5
#filename="test.txt"
filename="input.txt"
DIAG = False
DIAG2 = False

In [53]:
gearmap,gearcounts = parseFile(filename)
print("Size of gear map:",len(gearmap)) #dealWithRow(gearmap[0],gearcounts[0])

Size of gear map: 1000


In [54]:
def outerCountWithMultiplier(gearmap_in,gearcounts_in,multipl):
    gearmap2 = []
    gearcounts2 = []
    for i,line in enumerate(gearmap_in):
        new_line = ((line + "?")*multipl)[:-1]
        gearmap2.append(new_line)
        gearcounts2.append(gearcounts_in[i]*multipl)

    gearmap2= touchUpMap(gearmap2)

    DICT_BLANKS = {}
    res = 0

    for i,line in enumerate(gearmap2):
        if DIAG2: print("Line ",i,end=" ")
        res_line = dealWithRow(line,gearcounts2[i])
        if DIAG2: print(res_line)
        #print("line {} result={}".format(i,res_lddine))
        res += res_line
    if DIAG: print("Overall result:",res)

    return res

#gearmap2

In [55]:
resA = outerCountWithMultiplier(gearmap,gearcounts,1)
resB = outerCountWithMultiplier(gearmap,gearcounts,5)

print("Result part A: ",resA)
print("Result part B: ",resB)

Result part A:  7173
Result part B:  29826669191291
